# First Project for the Social Computing Course

We are gonna start to import all the stuff we need

In [ ]:
import tweepy
import os
import json
import pprint
import csv

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams

Now to use the API of Twitter, we need an developer account.  
So please create one and add the keys to continue use this notebook.

In [ ]:
# Twitter Credentials
api_key = input("Enter your api key: ")
api_key_secret = input("Enter your api key secret: ")
access_token = input("Enter your access token: ")
access_token_secret = input("Enter your access token secret: ")
bearer_token = input("Enter your bearer token: ")

Once you have done, it is time to create the wrapper of Tweepy to make requestes to the API

In [ ]:
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
if (api.verify_credentials):
  print("Authentication success!")
else:
  print("Got a error")

Authentication success!


So now we need to interact with the api and save the informations about the followers and the following.

In [ ]:
# Procedura
# Dato in input uno utente, crea due dataframe: il primo contiene l'insieme degli utenti trovati e il secondo le relazioni tra di loro

def saveUsersInformation(user, df_users, df_relations): 
    quantity = 500

    # request of the user
    user_info = api.get_user(screen_name=user)._json
    row = pd.json_normalize(user_info)
    df_users = pd.concat([df_users, row])

    user_id = user_info["id"]

    # request of user's followers
    for item in tweepy.Cursor(api.followers, id=user).items(quantity):
        data_item = item._json
        row_user = pd.json_normalize(data_item)
        # item -> user
        row_relation = pd.DataFrame([[data_item["id"], user_id]], columns=["Follower", "Following"])
        df_users = pd.concat([df_users, row_user])
        df_relations = pd.concat([df_relations, row_relation])

    # request of user's followings
    for item in tweepy.Cursor(api.friends, id=user).items(quantity):

      data_item = item._json
      row_user = pd.json_normalize(data_item)
      # user -> item
      row_relation = pd.DataFrame([[user_id, data_item["id"]]], columns=["Follower", "Following"])
      df_users = pd.concat([df_users, row_user])
      df_relations = pd.concat([df_relations, row_relation])

    return df_users, df_relations
